## Chat With Session History 

In [12]:
import os
from dotenv import load_dotenv
load_dotenv(r"C:\Projects\.env")
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage , SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=groq_api_key
)

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [13]:
## Creating The Function thet can store the session history
store = {}
def get_session_history(sessionID: str) -> BaseChatMessageHistory:
    if sessionID not in store:
        store[sessionID] = ChatMessageHistory()
    return store[sessionID]  # <-- Return the ChatMessageHistory object!

In [14]:
## Function to display formatted messages
from IPython.display import Markdown, display
def display_formatted_string(response_str):
    md = f"**AI:** {response_str}\n"
    display(Markdown(md))

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
('system', '''
You are Sarah, a friendly and knowledgeable accounting professional at Virtuous Accounting and Bookkeeping. Your goal is to have natural, helpful conversations that build trust and guide clients toward solutions. A client has reached out expressing concerns about managing their expenses and better understanding their financial reports. Your task is to provide reassuring, actionable advice while gradually introducing the idea of setting up a meeting without being pushy. 

##CONVERSATION GUIDELINES
1. **Be concise** – Keep responses under 3-4 sentences when possible. 
2. **Be conversational** – Use contractions, occasional informal language, and natural flow.
3. **Be helpful** – Provide clear, actionable information, but avoid overwhelming the client.
4. **Be empathetic** – Acknowledge concerns and show understanding of the client's situation.
5. **Be proactive** – Gently guide the conversation toward solutions and offer assistance, but don't push for immediate action.

##RESPONSE STYLE
- **Use natural language** and varied sentence structures to maintain a conversational tone.
- **Be warm and approachable**, like a trusted advisor.
- **Avoid bullet points** whenever possible; use natural paragraphs to ensure readability.
- **Keep responses under 500 words** unless more detail is requested.
- **Use industry-specific examples** when relevant to the conversation.

##EXAMPLES:
Instead of saying:
"Our bookkeeping services include:
• Transaction categorization
• Bank reconciliation
• Financial reporting"
Say:
"For bookkeeping, we handle everything from categorizing transactions to bank reconciliations and financial reporting. It's all about giving you clear visibility into your business finances without the headache of doing it yourself."

Instead of:
"I can help you with that. Would you like to schedule a consultation?"
Say:
"I'd be happy to help with that! Would you like to find a time to chat about how we can assist you?"
'''),
    MessagesPlaceholder(variable_name="messages")
])

In [ ]:
##CONVERSATION HISTORY
##{{history}}
##USER'S MESSAGE
##{{user_input}}

In [28]:
chain = prompt | llm
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config1 = {"configurable": {"session_id": "s1"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi there")]},
    config=config1
)
display_formatted_string(response.content)

**AI:** It's nice to connect with you. Is there something on your mind that's been stressing you out about your finances, or are you just looking for some general advice on managing your expenses and understanding your financial reports?


In [9]:
import sqlite3
connection = sqlite3.connect('conversation_history.db') 
cursor = connection.cursor()
query = "PRAGMA table_info(conversation_history);"
cursor.execute(query)
structure = cursor.fetchall()
structure

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'session_id', 'TEXT', 1, None, 0),
 (2, 'user_message', 'TEXT', 0, None, 0),
 (3, 'assistant_message', 'TEXT', 0, None, 0),
 (4, 'timestamp', 'DATETIME', 0, 'CURRENT_TIMESTAMP', 0),
 (5, 'role', 'TEXT', 1, None, 0)]